<br></br>
# **자연어와 Deep Learning**
## **LSTM 단어 알파벳 완성모델**

<br></br>
## **1 Data : 학습대상**
1. **X (Input Data)** : Sequence Data (전체 4개 알파벳의 연속성을 학습)
1. **Y (Output Data)** : Sequence Data (전체 2개 한글의 연속성을 학습)
1. **Sequence Data 를 Output** 으로 연결하기 위해 서로 다른 **RNN/ LSTM Cell을 덧붙인다**

In [1]:
# 분석할 데이터 (원하는 다른 내용으로 변경가능) 
seq_data = [['word', '단어'], ['wood', '나무'], ['game', '놀이'], 
            ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

<br></br>
## **2 데이터 임배딩 객체/ 함수(Batch 함수) 정의**
one-hot-encoding 활용

In [2]:
# Train 에 사용할 Input / Output 데이터를 숫자로 변환하기 위한 Index 를 특정한다 
char_arr = [ c for c in 'SPabcdefghijklmnopqrstuvwxyz나놀녀단랑무사소스어이키E'] 
num_dic  = { n : i for i, n in enumerate(char_arr)}
dic_len  = len(num_dic)

print("One-Hot-Encoding 객체의 수 : {}개\nOne-Hot-Encoding 내용 :\n{}".format(
    dic_len, num_dic))

One-Hot-Encoding 객체의 수 : 41개
One-Hot-Encoding 내용 :
{'S': 0, 'P': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27, '나': 28, '놀': 29, '녀': 30, '단': 31, '랑': 32, '무': 33, '사': 34, '소': 35, '스': 36, '어': 37, '이': 38, '키': 39, 'E': 40}


In [3]:
# Tensorflow 에 입력을 위한 Batch 함수를 정의한다
def make_batch(seq_data):
    input_batch, output_batch, target_batch = [], [], []
    for seq in seq_data:
        input_data = [num_dic[n] for n in seq[0]]
        output = [num_dic[n] for n in ('S' + seq[1])] 
        target = [num_dic[n] for n in (seq[1] + 'E')]
        input_batch.append(np.eye(dic_len)[input_data])
        output_batch.append(np.eye(dic_len)[output])
        target_batch.append(target)
    return input_batch, output_batch, target_batch

<br></br>
## **3 모델의 정의**

In [4]:
# 학습에 필요한 모듈을 불러온다
import tensorflow as tf
import numpy as np

In [5]:
# 학습에 필요한 Parameter
tf.reset_default_graph()
learning_rate = 0.01
n_hidden, total_epoch = 128, 100
n_class = n_input = dic_len

# Encoder / Decoder / Target 을 각각 정의한다
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets   = tf.placeholder(tf.int64, [None, None])  # [batch size, time steps]

In [6]:
# RNN Encoder Cell을 정의한다
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob = 0.5) 
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)

In [7]:
# RNN Decoder Cell을 정의한다
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, 
                                            initial_state = enc_states, 
                                            dtype = tf.float32)

In [8]:
# Cell 을 활용한 Model (Graph) 를 정의한다
# Cost 함수, Optimizer 함수를 정의한다
model = tf.layers.dense(outputs, n_class, activation=None)
cost  = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits = model, labels = targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

<br></br>
## **4 모델의 학습**
위에서 정의한 Model 에, **make_batch** 로 단어를 숫자로 변환 후 학습한다

In [9]:
%%time
sess = tf.Session()
sess.run(tf.global_variables_initializer())
input_batch, output_batch, target_batch = make_batch(seq_data)
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 9 == 0 :
        print('Epoch: {:4d}  cost = {:.6f}'.format((epoch + 1),loss))
print('최적화 완료!')

Epoch:    1  cost = 3.731578
Epoch:   10  cost = 0.147717
Epoch:   19  cost = 0.072504
Epoch:   28  cost = 0.014951
Epoch:   37  cost = 0.003715
Epoch:   46  cost = 0.000961
Epoch:   55  cost = 0.001373
Epoch:   64  cost = 0.000389
Epoch:   73  cost = 0.000334
Epoch:   82  cost = 0.000360
Epoch:   91  cost = 0.000355
Epoch:  100  cost = 0.000688
최적화 완료!
CPU times: user 1.5 s, sys: 395 ms, total: 1.9 s
Wall time: 1.49 s


<br></br>
## **5 모델의 검증**
seq_data 를 활용하여 모델을 검증한다

In [10]:
%%time
def translate(word):
    seq_data   = [word, 'P' * len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])
    prediction = tf.argmax(model, 2)   # [None, None, n_input]
    result     = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})
    decoded    = [char_arr[i] for i in result[0]]
    end        = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.68 µs


In [11]:
print('\n=== 번역 테스트 ===')
print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))
sess.close()


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 키스
